In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import os
#pd.set_option('display.max_rows', 200)

In [2]:
data = pd.read_excel('..\\Datos Partidos\\Aldosivi_Completo.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Session Start Date   38 non-null     datetime64[ns]
 1   Event                38 non-null     object        
 2   Session Name         38 non-null     object        
 3   Session Start        38 non-null     object        
 4   Session End          38 non-null     object        
 5   Tag Description      38 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            38 non-null     object        
 8   Tag End              38 non-null     object        
 9   Tag Duration (secs)  38 non-null     float64       
 10  Categoria            38 non-null     object        
 11  Zona2                38 non-null     object        
 12  Zona                 38 non-null     object        
 13  Accion               38 non-null     

In [3]:
data.head()

,Session Start Date,Event,Session Name,Session Start,Session End,Tag Description,Tag Notes,Tag Start,Tag End,Tag Duration (secs),Categoria,Zona2,Zona,Accion,Jugada
0,2022-06-12,Aldosivi,Completo,19:27:18,21:36:36,A FAVOR,NaN,00:28:36,00:28:38,2000.0,TRANSICION,5.0,6.0,VOLVER,1.0
1,2022-06-12,Aldosivi,Completo,19:27:18,21:36:36,A FAVOR,NaN,00:28:44,00:28:46,2000.0,TRANSICION,8A,5.0,VOLVER,1.0
2,2022-06-12,Aldosivi,Completo,19:27:18,21:36:36,A FAVOR,NaN,00:28:55,00:28:57,2000.0,TRANSICION,8B,8A,REMATE AFUERA,1.0
3,2022-06-12,Aldosivi,Completo,19:27:18,21:36:36,A FAVOR,NaN,00:31:10,00:31:12,2000.0,TRANSICION,7.0,4.0,VOLVER,2.0
4,2022-06-12,Aldosivi,Completo,19:27:18,21:36:36,A FAVOR,NaN,00:31:21,00:31:23,2000.0,TRANSICION,8B,7.0,VOLVER,2.0


In [4]:
data.columns = ['Fecha','etiquetado','name','Session Start','Session End','Evento','Tag Notes','Tag Start','Tag End','Tag Duration (secs)','Categoria','Zona inicial','Zona Final','Accion','Jugada']

In [5]:
'''CORREGIR MAYUSCULAS Y COSAS DE EVENTOS'''

data['Evento'] = data['Evento'].str.lower().str.strip()
data['Categoria'] = data['Categoria'].str.lower().str.strip()
data['Accion'] = data['Accion'].str.lower().str.strip()

## Estadisticas 

In [6]:
data.groupby(['Evento','Categoria'])['Jugada'].nunique().to_frame('Cantidad')

Cantidad
Evento    Categoria                 
a favor   ataque combinado         1
          pelota parada            4
          transicion               2
en contra ataque directo           2
          pelota parada            1
          transicion               3

### Armo las coordenadas

In [7]:
data['Zona inicial'] = data['Zona inicial'].replace('2A',2.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('2B',2.2,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8A',8.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8B',8.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2A',2.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2B',2.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8A',8.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8B',8.2,regex=True)

In [8]:
data['Zona inicial'].value_counts()

8.2    11
2.2     7
5.0     5
7.0     3
8.1     2
6.0     2
3.0     2
1.0     2
2.1     2
9.0     2
Name: Zona inicial, dtype: int64

In [9]:
data['Zona Final'].value_counts()

5.0    5
8.2    5
6.0    4
4.0    4
7.0    4
3.0    4
8.1    3
2.2    3
9.0    3
1.0    2
2.1    1
Name: Zona Final, dtype: int64

In [10]:
map_zona_coord_x = {  
1:45,
2.1:70,
2.2:30,
3:45,
4:136,
5:136,
6:136,
7:227,
8.1:255,
8.2:207,
9:227
}

In [11]:
map_zona_coord_y = {  
1:135,
2.1:81,
2.2:81,
3:27,
4:135,
5:81,
6:27,
7:135,
8.1:81,
8.2:81,
9:81
}

In [12]:
data['coord.final.num'] = np.where(data['Zona Final'] == 'volver',None, data['Zona Final']).astype('float')


In [13]:
data['Zona inicial x'] = data['Zona inicial'].map(map_zona_coord_x)
data['Zona Final x'] = data['coord.final.num'].map(map_zona_coord_x)
data['Zona inicial y'] = data['Zona inicial'].map(map_zona_coord_y)
data['Zona Final y'] = data['coord.final.num'].map(map_zona_coord_y)
data.iloc[:,-5:]

,coord.final.num,Zona inicial x,Zona Final x,Zona inicial y,Zona Final y
0,6.0,136,136,81,27
1,5.0,255,136,81,81
2,8.1,207,255,81,81
3,4.0,227,136,135,135
4,7.0,207,227,81,135
5,8.2,207,207,81,81
6,5.0,227,136,135,81
7,7.0,207,227,81,135
8,8.2,207,207,81,81
9,7.0,207,227,81,135


In [14]:
# pasar en ambos formatos. Crear las dos cols originales q sean strings

data['coord.inicial'] = round(data['Zona inicial x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona inicial y'],0).astype('str').str.split('.').str[0]
data['coord.final'] = round(data['Zona Final x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona Final y'],0).astype('str').str.split('.').str[0]


In [15]:
data['coord.inicial'] = np.where(data['coord.inicial']=='nan,nan', None, data['coord.inicial'])
data['coord.final'] = np.where(data['coord.final']=='nan,nan', None, data['coord.final'])

In [16]:
data[['coord.inicial','coord.final']]

,coord.inicial,coord.final
0,"136,81","136,27"
1,"255,81","136,81"
2,"207,81","255,81"
3,"227,135","136,135"
4,"207,81","227,135"
5,"207,81","207,81"
6,"227,135","136,81"
7,"207,81","227,135"
8,"207,81","207,81"
9,"207,81","227,135"


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                38 non-null     datetime64[ns]
 1   etiquetado           38 non-null     object        
 2   name                 38 non-null     object        
 3   Session Start        38 non-null     object        
 4   Session End          38 non-null     object        
 5   Evento               38 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            38 non-null     object        
 8   Tag End              38 non-null     object        
 9   Tag Duration (secs)  38 non-null     float64       
 10  Categoria            38 non-null     object        
 11  Zona inicial         38 non-null     object        
 12  Zona Final           38 non-null     object        
 13  Accion               38 non-null     

In [18]:
data['Temporada'] = 'Temp 02-2022'
data['partido'] = 'Aldosivi'

In [73]:
data.to_excel("../Datos generados/reserva02-22-fecha2_aldosivi_situaciones.xlsx",index=False)